In [ ]:
import progressbar
import h5py
import pandas as pd
import glob
from datetime import datetime, timedelta
import numpy as np
import pytz
import warnings
import os
import pylab as plt
%matplotlib inline
from matplotlib.colors import LogNorm

import search_dispersion_events 
import importlib
importlib.reload(search_dispersion_events)

In [ ]:

def search(filename, save=False):
    importlib.reload(search_dispersion_events)
    
    # Do computation --------------------------------------------------
    fh = search_dispersion_events.read_file(filename)

    dEicdt_smooth, Eic = search_dispersion_events.estimate_log_Eic_smooth_derivative(fh)

    df_match, integrand, integral, upper_area_frac = search_dispersion_events.walk_and_integrate(
        fh, dEicdt_smooth, Eic, search_dispersion_events.DEFAULT_INTERVAL_LENGTH,
        return_integrand=True
    )
    
    # Do plotting --------------------------------------------------
    for _, row_match in df_match.iterrows():
        i = fh['t'].searchsorted(row_match.start_time)
        j = fh['t'].searchsorted(row_match.end_time)

        fig, axes = plt.subplots(2, 1, figsize=(18, 6), sharex=True)

        im = axes[0].pcolor(fh['t'][i:j], np.log10(fh['ch_energy']), fh['ion_d_ener'][:, i:j], 
                            norm=LogNorm(vmin=1e3, vmax=1e8), cmap='jet')
        plt.colorbar(im, ax=axes[0]).set_label('Log Energy Flux')
        plt.colorbar(im, ax=axes[1]).set_label('')

        axes[0].plot(fh['t'][i:j], Eic[i:j], 'b*-')
        axes[0].invert_yaxis()
        axes[0].set_ylabel('Log Energy [eV] - Ions')

        time_length = row_match.end_time - row_match.start_time
        fig.suptitle(f'{time_length.total_seconds() / 60:.1f} minutes : '
                     f'{row_match.start_time.isoformat()} - {row_match.end_time.isoformat()}', fontweight='bold')
        
        title = 'MLAT = (%.1f deg to %.1f deg)' % (fh['mlat'][i], fh['mlat'][j])
        title += ' Northward' if fh['mlat'][j] > fh['mlat'][i] else ' Southward'
        title += f' -- Integral {float(integral[i:j].max()):.2f} -- UpperFrac {float(upper_area_frac[i]):.2f}'
        axes[0].set_title(title)

        axes[1].fill_between(fh['t'][i:j], 0, integrand[i:j])
        axes[1].axhline(0, color='black', linestyle='dashed')
        axes[1].set_ylim([-.25, .25])
        axes[1].set_ylabel('D(t) [eV/s]')

        if save:
            out_name = 'plots/Dec20_2015_storm/'
            out_name += f'{os.path.basename(filename)}_'
            out_name += f"{row_match.start_time.isoformat()}_"
            out_name += f"{row_match.end_time.isoformat()}.png"
            
            plt.savefig(out_name)
            plt.close()
            df_match['filename'] = filename

    return df_match

In [ ]:
files = []
for i in range(18, 24):
    files.extend(glob.glob('data/Dec20_2015_storm/**/*201512%02d*.hdf5'%i, recursive=True))

files[:5]

In [ ]:

df_matches = []

for i, file in enumerate(files):
    print(f'Processing {i+1}/{len(files)} :: {file}')
    df_match = search(file, save=True)
    df_matches.append(df_match)

In [ ]:
df = pd.concat(df_matches)
df.to_csv('data/Dec20_2015_storm_events.csv', index=0)
df